In [ ]:
# Step 1: Import Necessary Libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
import re

from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# --- Constants and Configuration ---
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 2


In [ ]:
# Step 2: Create the classes for the images based on the folder names with different conditions in each folder

# Set path for the director of the data set to be used
DATA_DIR = '/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/'

# Set the desired image dimensions for resizing.
IMG_HEIGHT = 224
IMG_WIDTH = 224


def create_classes_from_dirs(target_path):
    """
    Scans a target path and dynamically creates Python classes based on the names
    of the directories found within it.

    Args:
        target_path (str): The absolute or relative path to the directory to scan.

    Returns:
        dict: A dictionary containing the names and class objects that were created.
              Returns an empty dictionary if the path does not exist or contains no directories.
    """
    print(f"Attempting to scan for directories in: '{target_path}'")

    # --- 1. Validate the path ---
    if not os.path.isdir(target_path):
        print(f"\nError: The specified path does not exist or is not a directory.")
        print("Please make sure the path is correct and accessible.")
        return {}

    created_classes = {}

    # --- 2. Find all sub-directories ---
    try:
        # Get a list of all entries in the target_path and filter for directories
        dir_names = [name for name in os.listdir(target_path)
                     if os.path.isdir(os.path.join(target_path, name))]
    except OSError as e:
        print(f"\nError: Could not access the path. Reason: {e}")
        return {}

    if not dir_names:
        print("\nNo sub-directories were found in the specified path.")
        return {}

    print(f"\nFound {len(dir_names)} directories. Creating corresponding classes...")

    # --- 3. Create a class for each directory ---
    for dir_name in dir_names:
        # Sanitize the directory name to make it a valid Python class name
        # (e.g., "diabetic retinopathy" -> "Diabetic_retinopathy")
        class_name = re.sub(r'[^0-9a-zA-Z_]', '_', dir_name).capitalize()

        # If the first character is not a letter, prepend 'C' for 'Class'
        if not class_name[0].isalpha():
            class_name = 'C' + class_name

        # Dynamically create the class using the type() function
        # Format: type(ClassName, (BaseClasses,), {attributes_and_methods})
        new_class = type(class_name, (object,), {
            '__doc__': f'Dynamically generated class from the "{dir_name}" directory.',
            'source_directory': dir_name,
            'file_count': len(os.listdir(os.path.join(target_path, dir_name)))
        })

        # Add the new class to the global scope of this script, making it accessible
        globals()[class_name] = new_class
        created_classes[class_name] = new_class
        print(f"- Created class: {class_name}")

    return created_classes


# --- Main execution block ---
if __name__ == "__main__":

    # Generate the classes
    dynamic_classes = create_classes_from_dirs(DATA_DIR)

    # --- Verification Step ---
    # This section demonstrates how to use the dynamically created classes.
    if dynamic_classes:
        print("\n--- Verification ---")
        print("The classes have been created and are now available for use.")
        print("Let's inspect the first class that was created:\n")

        # Get the name of the first class from the dictionary
        first_class_name = list(dynamic_classes.keys())[0]

        # Access the class from the global scope using its name
        FirstClass = globals()[first_class_name]

        # Create an instance of the class
        instance = FirstClass()

        # Print some information about the instance and its class
        print(f"Class Name: {first_class_name}")
        print(f"Instance created: {instance}")
        print(f"Instance type: {type(instance)}")
        print(f"Class docstring: {instance.__doc__}")
        print(f"Original directory name: '{instance.source_directory}'")
        print(f"Number of files in directory: {instance.file_count}")

    if dynamic_classes:
        class_names = list(dynamic_classes.keys())
        class_names.sort()
        print("\n--- List of All Class Names ---")
        print("A list called 'class_names' has been created with the names of all generated classes.")
        print(class_names)

In [ ]:
# Step 3: Load and Preprocess the Data
def load_data(data_dir, class_names, img_width, img_height):
    images = []
    labels = []
    print("Loading image data...")
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                img = cv2.imread(img_path)

                # --- THIS IS THE FIX ---
                # Convert from BGR (OpenCV) to RGB (Matplotlib)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # --- END OF FIX ---

                img = cv2.resize(img, (img_width, img_height))
                images.append(img)
                labels.append(class_name)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    print("Image data loaded successfully.")
    return np.array(images), np.array(labels)

# Load the initial data
images, labels = load_data(DATA_DIR, class_names, IMG_WIDTH, IMG_HEIGHT)

# Normalize pixel values
# images = images.astype('float32') / 255.0

# --- Model Definition ---
def create_ovr_model(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)):
    """Creates a binary classifier based on EfficientNetB0 for OvR."""
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(name="avg_pool"),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name="top_dropout"),
        # Final layer for binary classification (One-vs-Rest)
        layers.Dense(1, activation='sigmoid', name="pred")
    ])
    return model

In [ ]:
# Step 4: Display sample images
def show_sample_images(sample_images, sample_labels, class_names_map):
    plt.figure(figsize=(12, 120))
    for i in range(min(30, len(sample_images))): # Show up to 240 images
        plt.subplot(60, 4, i + 1)
        plt.imshow(sample_images[i])

        # --- START OF FIX ---
        label_value = sample_labels[i]

        # Check if the label is an integer (like 0, 1) or a string (like 'cataract')
        if isinstance(label_value, (int, np.integer)):
            # If it's an integer, use it as an index to look up the name
            plt.title(class_names_map[label_value])
        else:
            # If it's already a string, just use it directly
            plt.title(label_value)
        # --- END OF FIX ---

        plt.axis("off")
    plt.suptitle("Sample Images from Training Set", fontsize=16, y=0.9)
    plt.show()

In [ ]:
# NEW Step 5 --- One-vs-Rest (OvR) Training Loop ---
# This version includes class weights to handle data imbalance and a two-stage fine-tuning process.

INITIAL_EPOCHS = 10
FINE_TUNE_EPOCHS = 10
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

for class_name in class_names:
    print("\n" + "="*50)
    print(f"    Training Model for: {class_name} vs. Rest")
    print("="*50 + "\n")

    # 1. Create Binary Labels
    binary_labels = np.array([1 if label == class_name else 0 for label in labels])

    # 2. Split the Data
    (X_train_temp, X_test, y_train_temp, y_test) = train_test_split(
        images, binary_labels, test_size=0.20, stratify=binary_labels, random_state=42
    )
    (X_train, X_val, y_train, y_val) = train_test_split(
        X_train_temp, y_train_temp, test_size=0.25, stratify=y_train_temp, random_state=42
    )
    print(f"Data split complete for '{class_name}'.")

    # --- THE FIX: CALCULATE CLASS WEIGHTS ---
    # This balances the loss function, forcing the model to pay attention to the minority class.
    weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight = {0: weights[0], 1: weights[1]}
    print(f"Class weights for '{class_name}': {class_weight}")

    # 3. Create and Compile the Model for Stage 1
    ovr_model = create_ovr_model()
    ovr_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # --- Stage 1: Initial Training (with Class Weights) ---
    print("\n--- Starting Stage 1: Initial Training ---")
    history = ovr_model.fit(
        X_train, y_train,
        epochs=INITIAL_EPOCHS,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        class_weight=class_weight,  # <--- Apply the class weights here
        verbose=2
    )

    # --- Stage 2: Fine-Tuning ---
    print("\n--- Starting Stage 2: Fine-Tuning ---")
    ovr_model.layers[0].trainable = True # Unfreeze the base model

    # Unfreeze the top 30 layers for fine-tuning
    for layer in ovr_model.layers[0].layers[:-30]:
        layer.trainable = False

    # Re-compile with a very low learning rate for fine-tuning
    ovr_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Continue training (fine-tuning)
    history_fine = ovr_model.fit(
        X_train, y_train,
        epochs=TOTAL_EPOCHS,
        initial_epoch=INITIAL_EPOCHS, #initial_epoch=history.epoch[-1],
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        class_weight=class_weight, # <--- Also apply weights during fine-tuning
        verbose=2
    )

    # 4. Save and Evaluate the Final Model
    model_filename = f"efficientnet_ovr_{class_name}_finetuned.h5"
    ovr_model.save(model_filename)
    print(f"Model saved to {model_filename}")

    test_loss, test_acc = ovr_model.evaluate(X_test, y_test, verbose=0)
    print(f"\nFinal Test Accuracy for {class_name} vs. Rest: {test_acc:.2%}")
    print(f"\nFinal Test Loss for {class_name} vs. Rest: {test_loss:.2}")

print("\nAll models have been trained and fine-tuned successfully!")


    Training Model for: Cataract vs. Rest

Data split complete for 'Cataract'.
Class weights for 'Cataract': {0: 0.6564299424184261, 1: 2.098159509202454}

--- Starting Stage 1: Initial Training ---
Epoch 1/10
43/43 - 17s - 391ms/step - accuracy: 0.6966 - loss: 0.5833 - val_accuracy: 0.9081 - val_loss: 0.3902
Epoch 2/10
43/43 - 12s - 276ms/step - accuracy: 0.8604 - loss: 0.3717 - val_accuracy: 0.9365 - val_loss: 0.3111
Epoch 3/10
43/43 - 12s - 275ms/step - accuracy: 0.8860 - loss: 0.2668 - val_accuracy: 0.9365 - val_loss: 0.2356
Epoch 4/10
43/43 - 12s - 277ms/step - accuracy: 0.9189 - loss: 0.2236 - val_accuracy: 0.9519 - val_loss: 0.1991
Epoch 5/10
43/43 - 12s - 277ms/step - accuracy: 0.9269 - loss: 0.1930 - val_accuracy: 0.9650 - val_loss: 0.1809
Epoch 6/10
43/43 - 12s - 277ms/step - accuracy: 0.9240 - loss: 0.2052 - val_accuracy: 0.9650 - val_loss: 0.1549
Epoch 7/10
43/43 - 12s - 277ms/step - accuracy: 0.9247 - loss: 0.1790 - val_accuracy: 0.9584 - val_loss: 0.1376
Epoch 8/10
43/43

Model saved to efficientnet_ovr_Cataract_finetuned.h5

Final Test Accuracy for Cataract vs. Rest: 96.94%

Final Test Loss for Cataract vs. Rest: 10.26%

    Training Model for: Conjunctivitis vs. Rest

Data split complete for 'Conjunctivitis'.
Class weights for 'Conjunctivitis': {0: 0.588134135855546, 1: 3.3365853658536584}

--- Starting Stage 1: Initial Training ---
Epoch 1/10
43/43 - 17s - 384ms/step - accuracy: 0.6396 - loss: 0.5789 - val_accuracy: 0.8643 - val_loss: 0.4166
Epoch 2/10
43/43 - 12s - 283ms/step - accuracy: 0.8209 - loss: 0.3119 - val_accuracy: 0.9278 - val_loss: 0.2720
Epoch 3/10
43/43 - 12s - 284ms/step - accuracy: 0.8838 - loss: 0.2703 - val_accuracy: 0.9344 - val_loss: 0.2214
Epoch 4/10
43/43 - 12s - 280ms/step - accuracy: 0.8845 - loss: 0.2434 - val_accuracy: 0.9475 - val_loss: 0.1958
Epoch 5/10
43/43 - 12s - 283ms/step - accuracy: 0.9123 - loss: 0.2170 - val_accuracy: 0.9453 - val_loss: 0.1802
Epoch 6/10
43/43 - 12s - 283ms/step - accuracy: 0.9218 - loss: 0.1998 

In [ ]:

# Step 6 --- Inference Section: Making a Prediction on a New Image ---

def predict_image_ovr(image_path, model_dir='.'):
    """
    Loads a new image, preprocesses it, and predicts its class
    using the saved One-vs-Rest models.
    """
    # 1. Load the trained OvR models
    ovr_models = {}
    for class_name in class_names:
        #model_path = os.path.join(model_dir, f"efficientnet_ovr_{class_name}.h5")
        model_path = os.path.join(model_dir, f"efficientnet_ovr_{class_name}_finetuned.h5") # Fix to use finely tuned version
        if os.path.exists(model_path):
            ovr_models[class_name] = tf.keras.models.load_model(model_path)
        else:
            print(f"Warning: Model not found for class '{class_name}' at {model_path}")
            return

    # 2. Load and preprocess the new image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img_array = np.expand_dims(img, axis=0)  # Add batch dimension
    img_array = img_array.astype('float32')

    # 3. Get a prediction score from each binary classifier
    scores = {}
    for class_name, model in ovr_models.items():
        score = model.predict(img_array)[0][0]
        scores[class_name] = score

    # 4. Determine the final prediction
    # The class with the highest probability score is the winner.
    predicted_class = max(scores, key=scores.get)
    highest_score = scores[predicted_class]

    print("\n--- Prediction Results ---")
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Predicted Condition: {predicted_class.capitalize()}")
    print(f"Confidence Score: {highest_score:.2%}")
    print("\n--- Individual Model Scores ---")
    for class_name, score in sorted(scores.items(), key=lambda item: item[1], reverse=True):
        print(f"- {class_name.capitalize()}: {score:.2%}")

In [18]:

# Example of how to use the prediction function
# Make sure you have an image to test with, for example 'test_image.jpg'
predict_image_ovr('/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification//21.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 732ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 686ms/step

--- Prediction Results ---
Image: 21.jpeg
Predicted Condition: Eyelid
Confidence Score: 73.40%

--- Individual Model Scores ---
- Eyelid: 73.40%
- Cataract: 14.41%
- Conjunctivitis: 4.21%
- Normal: 3.67%
- Uveitis: 1.18%
